# Compute trade conditions proportions using the dedicated service - Python

### Overview

A trade is often given a trade condition tag to give information on that trade such as the marketplace where it happened, the type of trade, etc. Standard/Regular trades and Off-Market trades are the two mainly used trading conditions.

This sample enables to retrieve **on-demand** data by calling a dedicated tick trade condition service on a tick by tick data store. 

### Inputs/outputs
The sample requires instrument's identifier, date time intervals as per inputs. It returns the trade condition values, their quantities, their percents and the total number of trades.

### Services used
This sample uses *gRPC requests* in order to retrieve tick trade condition from the dedicated hosted service. The queried endpoint in this script are:
* *TickTradeConditionService*: to directly retrieve tick trade condition objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* as per display package
***

# Run trade conditions proportions sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly mplfinance

In [ ]:
pip install systemathics.apis==2.33.* --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
from datetime import timedelta
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick_analytics.v1.tick_trade_condition_statistics_pb2 as tick_trade_condition_statistics
import systemathics.apis.services.tick_analytics.v1.tick_trade_condition_statistics_pb2_grpc as tick_trade_condition_statistics_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request
To request *TickTradeConditionService*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
my_ticker = 'AAPL'
my_exchange = 'XNGS'

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
today = datetime.today()
start = today - timedelta(days=5)

date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 11, day =3), 
    end_date = date.Date(year = 2021, month = 11, day = 8)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 14, minutes = 30, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 21, minutes = 0, seconds = 0)
)

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and ensure that the reply is not empty:

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval]
)

In [ ]:
# generate tick bars request
my_request = tick_trade_condition_statistics.TickTradeConditionStatisticsRequest(
    identifier = identifier.Identifier(exchange = my_exchange, ticker = my_ticker),
    constraints = my_constraints
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick bars service
        service = tick_trade_condition_statistics_service.TickTradeConditionStatisticsServiceStub(channel)
        
        # process the tick bars request
        statistics = []
        my_metadata = [('authorization', token)]
        for stat in service.TickTradeConditionStatistics(request=my_request, metadata=my_metadata):
                statistics.append(stat)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

### Step 4: Visualize  data

#### 4.1 Retrieve trade condition data

In [ ]:
# create the pandas dataframe for indices' components and weights
trade_conditions = [tc.value for tc in statistics]
counts =[tc.count for tc in statistics]
percentages = [tc.percentage*100 for tc in statistics]

d = {'Trading Condition': trade_conditions, 'Count': counts, 'Percentage': percentages }
df = pd.DataFrame(data=d)
df = df.sort_values(by = 'Count', ascending = False)
df.style.set_table_attributes("style='display:inline'").set_caption(("{0}-{1} Trading Conditions").format(my_ticker,my_exchange))

You can then use the `TickConditions` service to map each trading condition value to its description (*[Tick] conditions.ipynb* notebook).

In our case it appears the main trade conditions on this specific exchange and period are divided as follows:
* **9** (52%) : Odd Lot Trade
* **0** (28%) : Regular sale
* **6** (20%) : Intermarket Sweep

#### 4.2 Plot trade conditions with plotly

In [ ]:
import plotly.express as px

fig = px.pie(df, values='Percentage', names='Trading Condition',
             title = ('Trading Conditions Proportion for {0}-{1} ({2} trades between {3} and {4})').format(my_ticker,my_exchange,statistics[0].total,start.date(),today.date()))
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide', width=1000, height=500)
fig.show()